**<h1>Image Classification</h1>**

**<h3>Packages</h3>**

In [ ]:
import os
import cv2
from matplotlib import pyplot as plt
import matplotlib.patches as patches
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import pandas as pd
import seaborn as sn

In [ ]:
# from tensorflow.python.keras.applications.resnet50 import ResNet50
# from tensorflow.python.keras.applications.vgg16 import VGG16
# from tensorflow.python.keras.preprocessing import image
# from tensorflow.python.keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
import numpy as np

**<h3>Loading Images</h3>**

In [ ]:
import matplotlib.pyplot as plt
images = {}
all_image_paths = []
for x in os.listdir("images"):
    images[x] = Image.open("images/" + x)
    all_image_paths.append("images/" + x)
    newsize = (224,224)
    # display(images[x].resize(newsize))
    im_array = np.asarray(images[x].resize(newsize))
    plt.imshow(im_array)
    plt.show()

**<h3>Splitting Image Into Segments</h3>**

In [ ]:
def split_img(img, num_rows, num_cols, newsize=(224, 224)):
    # Get the dimensions of the original image
    img_width, img_height = img.size

    # Calculate the width and height of each ROI
    roi_width = img_width // num_cols
    roi_height = img_height // num_rows

    segments = {}

    for i in range(num_rows):
        for j in range(num_cols):
            # Calculate the coordinates for cropping
            left = j * roi_width
            upper = i * roi_height
            right = left + roi_width
            lower = upper + roi_height

            # Crop and resize the ROI
            roi = img.crop((left, upper, right, lower)).resize(newsize)
            segments[i, j] = roi

    return segments

In [ ]:
img = images[list(images.keys())[0]]
num_rows = 4
num_cols = 4

segments = split_img(img, num_rows, num_cols)
for segment in segments.values():
    display(segment)

**<h3>Loading Pretrained Weights</h3>**

In [ ]:
modelRN50 = ResNet50(weights='imagenet')
modelV16 = VGG16(weights='imagenet')

**<h3>Making Predictions</h3>**

In [ ]:
def predict_paths(model, path, target_size):
    predictions = []
    for i in range(len(path)):
        # Loading Image
        img = image.load_img(path[i], target_size=target_size)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        
        prediction = model.predict(x)
        print('Predicted:', decode_predictions(prediction, top=3)[0])
        predictions.append(prediction)
    
    return predictions

In [ ]:
def predict_imgs(model, images, target_size):
    predictions = []
    for key, value in images.items():
        x = images[key].resize(target_size)
        x = image.img_to_array(x)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        
        prediction = model.predict(x)
        print('Predicted:', decode_predictions(prediction, top=3)[0])
        predictions.append(prediction)
    
    return predictions

In [ ]:
resultRN50 = predict_imgs(modelRN50, segments, (224, 224))

In [ ]:
resultV16 = predict_imgs(modelV16, segments, (224, 224))

In [ ]:
def print_results(image_dict, result):
    images = {}
    i = 0
    for key, image in image_dict.items():
        print("Image {}: ".format(i+1))
        display(image)
        pred = decode_predictions(result[i], top=5)[0]
        print("Image {} 's top 5 predictions are: ".format(i+1))
        for j in range(len(pred)):
            print(str(j+1) + ")", pred[j][1], pred[j][2])
        print(" ")
        i+=1

**<h3>Displaying Results</h3>**

In [ ]:
def display_results(image_dict, resultRN50, resultV16):
    count = 0
    for key, image in image_dict.items():
        plt.figure(figsize=(10, 7))
        plt.rcParams['figure.figsize'] = (10, 7) 
        plt.rcParams['axes.titlesize'] = 20
        plt.subplot(1, 2, 1)
        plt.title("ResNet50")
        
        resnet_img = image.copy()
        resnet_pred = decode_predictions(resultRN50[count], top=5)[0]

        # Display the image
        plt.imshow(resnet_img)
        
        # Create a bounding box around the whole image
        ax = plt.gca()
        rect = patches.Rectangle((0, 0), resnet_img.size[1], resnet_img.size[0], linewidth=3, edgecolor='g', facecolor='none')
        ax.add_patch(rect)

        # Add the best prediction text to the image with a green background and white font
        _, best_label, best_score = resnet_pred[0]
        annotation_text = f'{best_label}: {best_score:.2%}'
        plt.annotate(annotation_text, xy=(10, 10), fontsize=16, color='white', backgroundcolor='green')

        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title("VGG16")
        plt.imshow(image)
        
        # Create a bounding box around the whole image for VGG16
        ax2 = plt.gca()
        rect2 = patches.Rectangle((0, 0), image.size[1], image.size[0], linewidth=3, edgecolor='g', facecolor='none')
        ax2.add_patch(rect2)

        # Add the best prediction text to the VGG16 image with a green background and white font
        vgg_img = image.copy()
        vgg_pred = decode_predictions(resultV16[count], top=5)[0]
        _, best_label_vgg, best_score_vgg = vgg_pred[0]
        annotation_text_vgg = f'{best_label_vgg}: {best_score_vgg:.2%}'
        plt.annotate(annotation_text_vgg, xy=(10, 10), fontsize=16, color='white', backgroundcolor='green')

        plt.axis('off')
        plt.show()
        count += 1

In [ ]:
display_results(segments, resultRN50, resultV16)

In [ ]:
print_results(segments, resultRN50)

In [ ]:
print_results(segments, resultV16)